# Using Anyscale Endpoints to create a Streaming Chatbot

In [ ]:
#!pip3 install -q openai (openai > 1.0)
!pip show openai

### 1. Define your Chat Agent 

In [1]:
import openai

class OpenAIChatAgent:
    def __init__(self, model: str):
    #In this simple example, we do not modify the past conversation.
    #Eventually you will run out of context window, but this should be enough for a 30-step conversation
    #You need to either trim the message history or summarize it for longer conversations
        self.message_history = []
        self.model = model

    def process_input(self, input: str):
        self.update_message_history(input)      
        
        openai.base_url = "https://api.endpoints.anyscale.com/v1" +'/'
        openai.api_key = ANYSCALE_ENDPOINT_TOKEN
                
        response = openai.chat.completions.create(
           model = self.model,
           messages = self.message_history,
           stream = True
        )
        words = ''
        for tok in response:
            delta = tok.choices[0].delta          
            if not delta: # End token 
                self.message_history.append({
                    'role': 'assistant',
                    'content': words
                })
                break
            elif delta.content:
                words += delta.content
                yield delta.content 
            else: 
                continue

    def update_message_history(self, inp):
        self.message_history.append({
            'role': 'user',
            'content': inp
        })

### 2. Instantiate a chat session

In [ ]:
import sys

ANYSCALE_ENDPOINT_TOKEN = "esecret_xxxxxxxxxxxxxxxxxx"

agent = OpenAIChatAgent("meta-llama/Llama-2-70b-chat-hf")
sys.stdout.write("Let's have a chat. (Enter `quit` to exit)\n") 
while True: 
    sys.stdout.write('> ')
    inp = input()
    if inp == 'quit':
        break
    for word in agent.process_input(inp):
        sys.stdout.write(word)
        sys.stdout.flush()
    sys.stdout.write('\n')